# Modeling

This notebook contains code that creates and tunes multiple classification models utilizing Random Forest (RF), Scalar Vector Machine (SVM), XGradient boost (XGB), and Multi-layer Perceptrons (MLP) algorithms. The final model is a voting classifier that utilizes each model as an estimator, where each estimator votes on the probability that a given field contains a particular crop. Ideally, this yields an ensemble model with greater predictive power than any single estimator could achieve working solo.

## Setup

Import the libraries + set the seed val

In [13]:
import pickle
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

import sys
sys.path.append('..')

from utils.file_ops import *
from utils.runtime_helpers import *

seed_val = 922

# get config
CONFIG = read_yaml('../../conf.yaml')

# get crops for crop_id -> crop_name resolution
crops = CONFIG['CROPS']

Load the data and add Crop Labels to Train Data

In [3]:
data_dir = '../../data/'

train_labels = pd.read_csv(f'{data_dir}labels_TRAIN.csv', index_col=[0])
train_data_agg = pd.read_csv(f'{data_dir}pixel_data_agg_TRAIN.csv', index_col=[0])
test_data_agg = pd.read_csv(f'{data_dir}pixel_data_agg_TEST.csv', index_col=[0])

train_data_and_labels = train_data_agg.merge(train_labels, on=['field_id'])
print(train_data_and_labels.shape)
train_data_and_labels.head()

(5551, 115)


,field_id,pixels,B01_median,B01_mean,B01_std,B01_range,B02_median,B02_mean,B02_std,B02_range,...,NDMI_range,NDWI_median,NDWI_mean,NDWI_std,NDWI_range,brightness_median,brightness_mean,brightness_std,brightness_range,crop_id
0,1,18,45.0,45.000000,0.000000,0,42.0,42.444444,0.511310,1,...,0.032660,-0.169811,-0.170352,0.009109,0.035951,63.5,63.688889,0.660560,2.0,1
1,2,12,45.0,45.000000,0.000000,0,42.0,42.000000,0.738549,2,...,0.040125,-0.201852,-0.205101,0.013323,0.042090,64.7,64.475000,0.903654,2.8,1
2,3,16,45.0,45.000000,0.000000,0,43.0,42.687500,1.250000,5,...,0.045760,-0.210526,-0.210190,0.017420,0.063147,66.5,66.543750,1.684822,5.1,1
3,4,15,46.0,45.866667,0.351866,1,43.0,42.466667,0.915475,3,...,0.027042,-0.166667,-0.177020,0.023648,0.076416,64.3,63.713333,1.084216,3.2,2
4,5,42,46.0,46.000000,0.000000,0,43.0,43.238095,0.576344,2,...,0.034662,-0.163636,-0.166941,0.015942,0.062861,64.6,64.704762,0.474189,2.3,2


## Create the Train / Test Splits

While the Radiant MLHub's AgriFieldNet Competition Dataset ships with test data, the test dataset does not contain crop ids, our predictor (y) values. In the following code block, we split the training data into Test/Train splits so we may assess (validate) the effectiveness of our models on unseen data.

We know this dataset is highly unbalanced, so we will utilize stratification to ensure each split contains data that matches the origin data's distribution of crop types. 

In [4]:
X = train_data_and_labels.drop(['field_id', 'crop_id'], axis=1)
y = train_data_and_labels['crop_id']

# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.33,
    random_state=seed_val,
    stratify=y
)

# save the collection of field ids
field_ids = test_data_agg['field_id']

# encode the labels (needed by XGB)
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

print(X_train.shape)
print(X_test.shape)

(3719, 113)
(1832, 113)


## Baseline Model

Radiant Earth includes a [starter notebook](https://github.com/radiantearth/agrifieldnet_india_competition/blob/main/Starter%20notebook.ipynb) that contains a baseline model for the AgrifieldNet India Competition. We can recreate and score that model using the following code.


In [39]:
baseline_agg_idxs = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']
baseline_agg_metrics = [ '_mean' ]
baseline_selected = get_features(baseline_agg_idxs, baseline_agg_metrics)

baseline_model = RandomForestClassifier(
    n_estimators=20, 
    random_state=0, 
    n_jobs=3)
baseline_model.fit(X_train[baseline_selected], y_train)

(
    baseline_model_y_prob, baseline_model_y_pred, baseline_model_acc, baseline_model_auc, baseline_model_ll
) = eval_and_print_metrics( baseline_model, X_train[baseline_selected], y_train, X_test[baseline_selected], y_test )

Accuracy: 0.6277292576419214
ROC AUC: 0.7732696502968152
Logloss: 2.5656738792218037
              precision    recall  f1-score   support

           0       0.63      0.80      0.71       670
           1       0.42      0.33      0.37       327
           2       0.00      0.00      0.00        34
           3       0.75      0.77      0.76       542
           4       0.00      0.00      0.00         8
           5       0.15      0.07      0.10        54
           6       0.00      0.00      0.00        16
           7       0.72      0.85      0.78        97
           8       0.14      0.05      0.08        19
           9       0.00      0.00      0.00         5
          10       0.00      0.00      0.00        13
          11       0.00      0.00      0.00         5
          12       0.20      0.07      0.11        42

    accuracy                           0.63      1832
   macro avg       0.23      0.23      0.22      1832
weighted avg       0.58      0.63      0.60      

The results for our baseline model agree with Radiant Earth's baseline model acheiving approximately 61% accuracy, 74% AUC, and a logloss of 2.57. Note that the logloss calculated above agrees with the competition benchmark found on the [leaderboard](https://zindi.africa/competitions/agrifieldnet-india-challenge/leaderboard) which is 2.564949357.

## Modeling

We will use pipelines to preprocess and model our data. Each model will utilize identical preprocessing steps composed of scaling, feature selection, and oversampling of minority classes. 

- scaling: given that our features contain potential outliers, we utilize the `RobustScaler` as it scales features using statistics robust to outliers.
- programmatic feature selection: `VarianceThreshold` removes low-variance features to improve the predictive power of our data (theoretically, at least.)
- oversampling: crop types are not equally represented (imbalanced). Using `SMOTE`, we will oversample the minority classes in the data to attempt to overcome this.

*Note: Past iterations also used PCA in the preprocessing pipeline, but we determined that PCA reduced our models' predictive power as implemented.*

Below we utilize GridSearch to perform hyperparameter tuning for each model. We are using log loss as our scoring metric as it is better suited for evaluation than the default scoring metric, accuracy, due to the imbalanced nature of our data. We will Then, we execute and save a (somewhat tuned) model.

### Random Forest

In [113]:

param_grid = {
    'classifier__n_estimators': [350, 400, 500],
    'classifier__max_depth': [30, 40, 50],
}

rf_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', RandomForestClassifier(random_state=seed_val))
])

rf_p_cv = GridSearchCV(
    estimator=rf_p,
    param_grid=param_grid, 
    scoring='neg_log_loss', 
    cv=4,
    verbose=False
).fit(X_train, y_train)

print(rf_p_cv.best_score_)
print(rf_p_cv.best_params_)

-1.2041574267974453
{'classifier__max_depth': 40, 'classifier__n_estimators': 500}


In [117]:
rf_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', RandomForestClassifier(
        random_state=seed_val,
        max_depth=40,
        n_estimators=500
        ))
])
rf_p.fit(X_train, y_train)

Pipeline(steps=[('scaler', RobustScaler()),
                ('selector', VarianceThreshold(threshold=0.001)),
                ('smote', SMOTE(random_state=922)),
                ('classifier',
                 RandomForestClassifier(max_depth=40, n_estimators=500,
                                        random_state=922))])

In [118]:
(
    rf_p_y_prob, rf_p_y_pred, rf_p_acc, rf_p_auc, rf_p_ll
) = eval_and_print_metrics( rf_p, X_train, y_train, X_test, y_test )

Accuracy: 0.62882096069869
ROC AUC: 0.8496045952173322
Logloss: 1.1828996408655292
              precision    recall  f1-score   support

           0       0.70      0.74      0.72       670
           1       0.45      0.37      0.41       327
           2       0.07      0.06      0.06        34
           3       0.86      0.74      0.80       542
           4       0.00      0.00      0.00         8
           5       0.22      0.43      0.29        54
           6       0.05      0.06      0.05        16
           7       0.63      0.88      0.73        97
           8       0.21      0.37      0.27        19
           9       0.00      0.00      0.00         5
          10       0.08      0.08      0.08        13
          11       0.00      0.00      0.00         5
          12       0.28      0.26      0.27        42

    accuracy                           0.63      1832
   macro avg       0.27      0.31      0.28      1832
weighted avg       0.64      0.63      0.63      18

In [123]:
filename='../../saved_models/rf_p.sav'
pickle.dump(rf_p, open(filename, 'wb'))

### X Gradient Boost

In [120]:

param_grid = {
    'classifier__n_estimators': [100, 120, 150],
    'classifier__max_depth': [10, 15, 20],
    'classifier__learning_rate': [0.09, 0.1, 0.2]
}

xgb_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', XGBClassifier(random_state=seed_val))
])

xgb_p_cv = GridSearchCV(
    estimator=xgb_p,
    param_grid=param_grid, 
    scoring='neg_log_loss',
    cv=4,
    verbose=False
).fit(X_train, y_train)

print(xgb_p_cv.best_score_)
print(xgb_p_cv.best_params_)

-1.16688357944116
{'classifier__learning_rate': 0.09, 'classifier__max_depth': 10, 'classifier__n_estimators': 100}


In [122]:
xgb_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', XGBClassifier(
        random_state=seed_val,
        learning_rate=0.09,
        max_depth=10,
        n_estimators=100))
])
xgb_p.fit(X_train, y_train)


Pipeline(steps=[('scaler', RobustScaler()),
                ('selector', VarianceThreshold(threshold=0.001)),
                ('smote', SMOTE(random_state=922)),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma...pu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='', learning_rate=0.09,
                               max_bin=256, max_cat_threshold=64,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=10, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto', ...))])

In [124]:
(
    xgb_p_y_prob, xgb_p_y_pred, xgb_p_acc, xgb_p_auc, xgb_p_ll
) = eval_and_print_metrics( xgb_p, X_train, y_train, X_test, y_test )

Accuracy: 0.6457423580786026
ROC AUC: 0.8504865032845426
Logloss: 1.0672059052116207
              precision    recall  f1-score   support

           0       0.69      0.80      0.74       670
           1       0.47      0.36      0.41       327
           2       0.04      0.03      0.04        34
           3       0.82      0.77      0.79       542
           4       0.00      0.00      0.00         8
           5       0.28      0.33      0.30        54
           6       0.00      0.00      0.00        16
           7       0.68      0.84      0.75        97
           8       0.14      0.16      0.15        19
           9       0.00      0.00      0.00         5
          10       0.08      0.08      0.08        13
          11       0.00      0.00      0.00         5
          12       0.33      0.26      0.29        42

    accuracy                           0.65      1832
   macro avg       0.27      0.28      0.27      1832
weighted avg       0.63      0.65      0.63      

In [125]:
filename='../../saved_models/xgb_p.sav'
pickle.dump(xgb_p, open(filename, 'wb'))

### Scalable Vector Machines

In [ ]:
param_grid = {
    'classifier__kernel': ['poly', 'rbf', 'sigmoid'],
    'classifier__decision_function_shape': ['ovr', 'ovo'],
}

svm_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', svm.SVC(
        random_state=seed_val,
        probability=True))
])

svm_p_cv = GridSearchCV(
    estimator=svm_p,
    param_grid=param_grid, 
    scoring='neg_log_loss', 
    cv=4,
    verbose=False
).fit(X_train, y_train)

print(svm_p_cv.best_score_)
print(svm_p_cv.best_params_)

-1.2792363772832647
{'classifier__decision_function_shape': 'ovr', 'classifier__kernel': 'rbf'}


In [ ]:
svm_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', svm.SVC(
        random_state=seed_val,
        probability=True,
        kernel='rbf',
        decision_function_shape='ovr',
        ))
])

svm_p.fit(X_train, y_train)

Pipeline(steps=[('scaler', RobustScaler()),
                ('selector', VarianceThreshold(threshold=0.001)),
                ('smote', SMOTE(random_state=922)),
                ('classifier', SVC(probability=True, random_state=922))])

In [ ]:
(
    svm_p_y_prob, svm_p_y_pred, svm_p_acc, svm_p_auc, svm_p_ll
) = eval_and_print_metrics( svm_p, X_train, y_train, X_test, y_test )

Accuracy: 0.5294759825327511
ROC AUC: 0.81495972641027
Logloss: 1.2665063802296515
              precision    recall  f1-score   support

           0       0.74      0.55      0.63       670
           1       0.45      0.34      0.39       327
           2       0.07      0.29      0.11        34
           3       0.87      0.61      0.72       542
           4       0.06      0.12      0.08         8
           5       0.19      0.57      0.29        54
           6       0.06      0.12      0.08        16
           7       0.64      0.85      0.73        97
           8       0.15      0.47      0.23        19
           9       0.00      0.00      0.00         5
          10       0.07      0.15      0.10        13
          11       0.00      0.00      0.00         5
          12       0.17      0.40      0.24        42

    accuracy                           0.53      1832
   macro avg       0.27      0.35      0.28      1832
weighted avg       0.65      0.53      0.57      18

In [ ]:
filename='../../saved_models/svm_p.sav'
pickle.dump(svm_p, open(filename, 'wb'))

### Multi-Layer Perceptron

In [49]:
np.arange(start=0.00008, stop=0.00011, step=0.00001)

array([8.e-05, 9.e-05, 1.e-04])

In [121]:
param_grid = {
    'classifier__hidden_layer_sizes': [8, 16, 32, 64],
    'classifier__solver': ['sgd', 'adam'],
    'classifier__alpha': np.arange(start=0.00008, stop=0.0005, step=0.0001),
}

mlp_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', MLPClassifier(
        random_state=seed_val,
        learning_rate='constant',
        max_iter=4000))
])

mlp_p_cv = GridSearchCV(
    estimator=mlp_p,
    param_grid=param_grid, 
    scoring='neg_log_loss', 
    cv=4,
    verbose=False
).fit(X_train, y_train)

print(mlp_p_cv.best_score_)
print(mlp_p_cv.best_params_)

-1.9405638743422327
{'classifier__alpha': 8e-05, 'classifier__hidden_layer_sizes': 8, 'classifier__solver': 'sgd'}


In [126]:
mlp_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', MLPClassifier(
        random_state=seed_val,
        learning_rate='constant',
        max_iter=4000,
        alpha=8e-05,
        hidden_layer_sizes=8,
        solver='sgd'
        ))
])
# {'alpha': 0.00038, 'hidden_layer_sizes': 32, 'learning_rate': 'constant', 'max_iter': 4000, 'solver': 'adam'}
mlp_p.fit(X_train, y_train)

Pipeline(steps=[('scaler', RobustScaler()),
                ('selector', VarianceThreshold(threshold=0.001)),
                ('smote', SMOTE(random_state=922)),
                ('classifier',
                 MLPClassifier(alpha=8e-05, hidden_layer_sizes=8, max_iter=4000,
                               random_state=922, solver='sgd'))])

In [127]:
(
    mlp_p_y_prob, mlp_p_y_pred, mlp_p_acc, mlp_p_auc, mlp_p_ll
) = eval_and_print_metrics( mlp_p, X_train, y_train, X_test, y_test )

Accuracy: 0.40010917030567683
ROC AUC: 0.7689052492828844
Logloss: 1.9805467485050607
              precision    recall  f1-score   support

           0       0.71      0.41      0.52       670
           1       0.43      0.25      0.32       327
           2       0.04      0.24      0.07        34
           3       0.81      0.43      0.56       542
           4       0.00      0.00      0.00         8
           5       0.18      0.61      0.28        54
           6       0.04      0.19      0.07        16
           7       0.51      0.76      0.61        97
           8       0.11      0.37      0.17        19
           9       0.00      0.00      0.00         5
          10       0.02      0.08      0.03        13
          11       0.00      0.00      0.00         5
          12       0.08      0.33      0.13        42

    accuracy                           0.40      1832
   macro avg       0.23      0.28      0.21      1832
weighted avg       0.61      0.40      0.46     

Accuracy: 0.537117903930131
ROC AUC: 0.7818578193718655
Logloss: 1.9727422551131373

In [128]:
filename='../../saved_models/mlp_p.sav'
pickle.dump(mlp_p, open(filename, 'wb'))

### Putting it all together (Voting Classifier)

In [129]:
estimators = [
    ('rf', RandomForestClassifier(
            random_state=seed_val,
            max_depth=40,
            n_estimators=500
        )),
    ('xgb', XGBClassifier(
            random_state=seed_val,
            learning_rate=0.09, 
            max_depth=10, 
            n_estimators=100
    )),
    ('svm', svm.SVC(
            random_state=seed_val,
            decision_function_shape='ovr',
            kernel='rbf',
            probability=True
        )),
    ('mlp', MLPClassifier(
            random_state=seed_val,
            learning_rate='constant', 
            max_iter= 4000, 
            alpha=8e-05,
            hidden_layer_sizes=8,
            solver='sgd'
        ))
]

vc_p = Pipeline([
    ('scaler', RobustScaler()),
    ('selector', VarianceThreshold(threshold=0.001)),
    ('smote', SMOTE(random_state=seed_val)),  
    ('classifier', VotingClassifier(estimators=estimators, voting='soft'))
])
vc_p.fit(X_train, y_train)

Pipeline(steps=[('scaler', RobustScaler()),
                ('selector', VarianceThreshold(threshold=0.001)),
                ('smote', SMOTE(random_state=922)),
                ('classifier',
                 VotingClassifier(estimators=[('rf',
                                               RandomForestClassifier(max_depth=40,
                                                                      n_estimators=500,
                                                                      random_state=922)),
                                              ('xgb',
                                               XGBClassifier(base_score=None,
                                                             booster=None,
                                                             callbacks=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=...
                                                             max_delta_step=None,
                                                             max_depth=10,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             n_estimators=100,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             predictor=None,
                                                             random_state=922, ...)),
                                              ('svm',
                                               SVC(probability=True,
                                                   random_state=922)),
                                              ('mlp',
                                               MLPClassifier(alpha=8e-05,
                                                             hidden_layer_sizes=8,
                                                             max_iter=4000,
                                                             random_state=922,
                                                             solver='sgd'))],
                                  voting='soft'))])

In [130]:
(
    vc_p_y_prob, vc_p_y_pred, vc_p_acc, vc_p_auc, vc_p_ll
) = eval_and_print_metrics( vc_p, X_train, y_train, X_test, y_test )

Accuracy: 0.62117903930131
ROC AUC: 0.8480485040921213
Logloss: 1.0730791037090281
              precision    recall  f1-score   support

           0       0.72      0.72      0.72       670
           1       0.48      0.36      0.41       327
           2       0.04      0.06      0.05        34
           3       0.85      0.73      0.79       542
           4       0.00      0.00      0.00         8
           5       0.24      0.59      0.34        54
           6       0.12      0.19      0.15        16
           7       0.61      0.88      0.72        97
           8       0.21      0.32      0.25        19
           9       0.00      0.00      0.00         5
          10       0.00      0.00      0.00        13
          11       0.00      0.00      0.00         5
          12       0.27      0.29      0.28        42

    accuracy                           0.62      1832
   macro avg       0.27      0.32      0.28      1832
weighted avg       0.65      0.62      0.63      18

In [131]:
filename = '../../saved_models/vc_p.sav'
pickle.dump(vc_p, open(filename, 'wb'))

## Conclusions and Next Steps

Above we created, tuned, and evaluated models utilizing Random Forest (RF), Scalar Vector Machine (SVM), XGradient boost (XGB), and Multi-layer Perceptrons (MLP) algorithms. The XGB and RF models performed the best solo. But we opted to then combine the predictive power of all 4 models using a voting classifier.

Looking at the classification report for each model, we see that some classes are easier to classify than others. 


In [18]:
crop_names = [get_crop_name_from_id(crops, cid) for cid in le.classes_]
le_resolution = pd.DataFrame(crop_names, columns=['crop_name'])
le_resolution.index.name = 'label'
le_resolution

,crop_name
label,
0,Wheat
1,Mustard
2,Lentil
3,No crop/Fallow
4,Green pea
5,Sugarcane
6,Garlic
7,Maize
8,Gram


Using the above resolution map alongside the classification reports, we find that some crops were easier to classify than others. Crops that were well represented in the dataset, wheat, mustard, maise, and no crop/fallow were classified with relatively good precision. Whereas crops that were relatively underrepresented in the dataset, lentils, greenpeas, garlic, coriander, potatoes, and berserm were classified with far lower precision. Even though we oversampled our minority classes, there were likely still too few instances of these classes to effectively create _useful_ synthetic data. 

In the next notebook we will refit our models, obtain predictions, and prepare the results for submission to the AgrifieldNet competition site.